In [2]:
# Kathleen Kish
# July 29, 2022
# Script to apply extracellular stimulus pulse to retinal ganglion cell

# Build Stimulus wave: biphasic, charge-balanced, cathodic-first current pulses with no interphase gap
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def stimulus(PW,frequency,tstop,dt): 
    # PW: msec; frequency: Hz; total_time, dt: msec
    time=np.arange(0,tstop,dt) # time vector is in msec
    time=np.around(time,decimals=6)
    time_between_stimuli=(np.float(1)/np.float(frequency))*(1e3) # convert frequency (Hz) to time between stimuli (msec)
    
    pulse_start_times=np.arange(10,tstop,time_between_stimuli) # in msec
    pulse_middle_times=np.arange(10+PW,tstop,time_between_stimuli) # in msec
    pulse_end_times=np.arange(10+(2*PW),tstop,time_between_stimuli) # in msec
    number_of_pulses=len(pulse_start_times)
    
    amplitudes=[]
    for i in np.arange(len(time)):
        amplitudes.append(0)
    
    start_indices=[]
    middle_indices=[]
    end_indices=[]
    for j in np.arange(number_of_pulses):
        s=find_nearest(time, pulse_start_times[j])
        start_indices.append(s)
        m=find_nearest(time, pulse_middle_times[j])
        middle_indices.append(m)
        e=find_nearest(time,pulse_end_times[j])
        end_indices.append(e)
    
    first_phase_indices=[]
    second_phase_indices=[]
    for k in np.arange(number_of_pulses):
        n=np.arange(start_indices[k],middle_indices[k],1)
        first_phase_indices.append(n)
        p=np.arange(middle_indices[k],end_indices[k],1)
        second_phase_indices.append(p)
    first_phase_indices_fixed=np.concatenate(first_phase_indices,axis=0)
    second_phase_indices_fixed=np.concatenate(second_phase_indices,axis=0)

    for (index, amplitude) in zip(first_phase_indices_fixed, amplitudes):
        amplitudes[index]= -1 # sign and relative amplitude of first-phase of stimulus pulse
    for (index, amplitude) in zip(second_phase_indices_fixed, amplitudes):
        amplitudes[index]= 1 # sign and relative amplitude of second-phase of stimulus pulse
    amplitudes[0]=0
                
    return(amplitudes)

# Foundational function that applies extracellular voltage to the neuron through time
def integrate(tstop, dt, pulse_shape, voltages, scaling_var):
    h.load_file("stdrun.hoc")
    h.init() # re-initialize cell at the start of every integration
    h.v_init = -70 # resting membrane potential
    h.finitialize(h.v_init) # initialize the starting voltage throughout the cell
    h.fcurrent() # initialize the membrane currents  
    h.cvode_active(0)
    h.tstop = tstop # total stimulation time (msec)
    h.dt = dt
    t = 0 # time indexing variable
    while round(h.t,4) < h.tstop:
        s = 0 # space indexing variable
        for sec in h.allsec():
            for seg in sec:
                seg.e_extracellular = pulse_shape[t]*voltages[s]*scaling_var
                s = s+1
        t = t+1
        h.fadvance()

# Load coordinate and voltage data from a .txt file
def load_txt_file(path_to_obj,dtype):
	output = open(path_to_obj,'r')
	out_file = list()
	
	for line in output:
		tmp = line.split()
		tmp1 = list()
		for ii in tmp:
			if dtype == 'float':
				tmp1.append(float(ii))
			elif dtype == 'int':
				tmp1.append(int(ii))
			else:
				tmp1.append(ii)
		out_file.append(tmp1)
	output.close()
	
	return out_file